In [181]:
# Dependencies
import requests
import json
import pandas as pd
import numpy as np
from pprint import pprint
from time import sleep

# Google developer API key
from api_keys import gkey_jr

In [176]:
# read zipcode file and create dataframe
zipcode_file = "../data/rawdata/Harris_County.csv"
lat_lng_file = "../data/rawdata/zipcode_lat_lngs.csv"
zip_codes = [77002, 77004, 77003, 77006, 77005, 77008, 77007, 77010, 
             77009, 77012, 77011, 77014, 77013, 77016, 77015, 77018, 
             77017, 77020, 77019, 77022, 77021, 77024, 77023, 77026, 
             77025, 77028, 77027, 77030, 77029, 77032, 77031, 77034, 
             77033, 77036, 77035, 77038, 77037, 77040, 77039, 77042, 
             77041, 77044, 77043, 77046, 77045, 77048, 77047, 77050, 
             77049, 77051, 77054, 77053, 77056, 77055, 77058, 77057, 
             77060, 77059, 77062, 77061, 77064, 77063, 77066, 77065, 
             77068, 77067, 77070, 77069, 77072, 77071, 77074, 77073, 
             77076, 77075, 77078, 77077, 77080, 77079, 77082, 77081, 
             77084, 77083, 77086, 77085, 77088, 77087, 77090, 77089, 
             77092, 77091, 77094, 77093, 77096, 77095, 77098, 77099, 
             77204, 77217, 77249, 77251, 77266, 77268, 77271, 77284, 
             77289, 77336, 77339, 77338, 77345, 77346, 77357, 77373, 
             77375, 77377, 77379, 77383, 77389, 77388, 77396, 77401, 
             77410, 77429, 77433, 77447, 77450, 77449, 77484, 77493, 
             77503, 77502, 77505, 77504, 77507, 77506, 77521, 77520, 
             77530, 77532, 77536, 77546, 77547, 77562, 77571, 77586, 77587, 77598]
lat_lng_df = pd.read_csv(lat_lng_file, encoding="ISO-8859-1")

#identify Harris zipcodes in lat_lng file
harris_df=pd.DataFrame()

i=0
for tzip in zip_codes:
    #harris_df.at[i,'zip code']=zip
    
    inner = lat_lng_df[lat_lng_df['ZIP']==tzip]
#     print(inner)
    harris_df = harris_df.append(inner)
    #harris_df.at[i,'lat']=harris_zip['LAT']
    i=i+1
harris_df.reset_index(inplace=True,drop=True)
harris_df.head()

,ZIP,LAT,LNG
0,77002,29.756845,-95.365652
1,77004,29.724893,-95.363752
2,77003,29.749778,-95.345885
3,77006,29.740970,-95.391301
4,77005,29.718435,-95.423555


In [177]:
ammenities_df = pd.DataFrame(data={"lat":"", "lng":"","# Restaurants":"" }, index = [i for i in range(len(harris_df))])
ammenities_df.head()

,lat,lng,# Restaurants
0,,,
1,,,
2,,,
3,,,
4,,,


In [206]:
# base url and query specifics
ammenities_df = pd.DataFrame(data={"lat":"", "lng":"","# Restaurants":"" }, index = [i for i in range(len(harris_df))])
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
target_radius = 500
target_type = "bar"

# geocoordinates
#for i in range(0,len(harris_df),1):
for i in range(1):
    lat=harris_df.iloc[i,1]
    lng=harris_df.iloc[i,2]
    target_coordinates = f"{lat}, {lng}"
    print(target_coordinates)

    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "type": target_type,
        "key": gkey_jr
    }

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    
    #count number of ammenities
    places_data = response.json()
    
    pprint(places_data)


29.756845000000002, -95.365652
{'html_attributions': [],
 'next_page_token': 'CqQCGgEAALZZH65Ow-vq22HDnuZy9LyazGjTMCruJi4RfHfHwb3sqMvhhfvE1iDbXTg0Pzimf-u7puXEaq-z_s8YuwOe18f8kaT_-c99kZC4gRCjTtDZsBvGj-jXZi5RrcIkPDZDKekGC8M1vZKSjwcA4YATy89MpK3B2MoqcfyBpsk6RZYqdqKyzMdyzRMX0s9FPo1DgSBiWA6qsXMM9E_i-0bh4GFBj4kYuqPipZfdPI2jrbwxLhfdlvGqRHvLpi9RiYJJv1aTMXorsCSHcG0BKTHEA951dAphfdhs2IIgt-EqH01raDd_ULOAL7inlXJrwshaOnwxZlcRxdQLi0ucFLCeIiGqtOueHSDbVGksCyWsnLN6EsEaMZpmwapGEnfANUGlEBIQkgTF_RwKHUjIox7OukrCUxoUnBAgagDUC3ZADn5wANi54lSBDk0',
 'results': [{'geometry': {'location': {'lat': 29.7536682,
                                        'lng': -95.36435349999999},
                           'viewport': {'northeast': {'lat': 29.7551393802915,
                                                      'lng': -95.36325116970849},
                                        'southwest': {'lat': 29.7524414197085,
                                                      'lng': -95.36594913029151}}},
              'icon':

In [154]:
page_token=places_data["next_page_token"]
print(page_token)
        
params = {
"key": gkey_jr,
"pagetoken":page_token
}  

response = requests.get(base_url, params=params)
places_data = response.json()
pprint(places_data)

CqQCGgEAANsYYon0sn6aekb2RqIwYetRAf79IZsbKe4a_GarPQ7I80qI5FlY5wY519TWg0xmYPjQRFzayoueJiD7oYa4BLDIz-m4edFme-LCzzWIVbbV3YWZn3LmPH388XiDoGP4QFmSc3M1vX7P-yww41QsopPrxxVT55CkUmC_MT8tNBtsvykKNLwp7faoIgfm3WRxP9VxpAZXcKdaWnhU29Q_m3ssPiNkNtLlaXFB8qdwGtigaftw8k5xxzA6DfvPRK27WPVwPMAS4uAG1SYjM_EMuCY4KkdSbuMcL6EgJiKG6XWGHHcztHZgn6uWwud7FUmhexz5Da43zzoX_oGynVk-TTbWGS7_RSkKzOOyytnQlHRPjTX8KuzKwJCZFcgMK4lZ4BIQ27rSD1vsISltQi6-kufxjBoU-tT4GvozQfQAKACEEhc7GX_mwhg
{'html_attributions': [],
 'results': [{'geometry': {'location': {'lat': 29.754811, 'lng': -95.3655795},
                           'viewport': {'northeast': {'lat': 29.7562761302915,
                                                      'lng': -95.36413751970849},
                                        'southwest': {'lat': 29.7535781697085,
                                                      'lng': -95.3668354802915}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/bar-71.png',
              'id': 'e505e8c9058f4e791

In [135]:
page_token=places_data["next_page_token"]
page_token

KeyError: 'next_page_token'

In [220]:
ammenities_df = pd.DataFrame(data={"lat":"", "lng":"","# Restaurants":"" }, index = [i for i in range(len(harris_df))])
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
target_radius = 1000
target_type = "restaurant"

# geocoordinates
#for i in range(0,len(harris_df),1):
for i in range(len(harris_df)):
    lat=harris_df.iloc[i,1]
    lng=harris_df.iloc[i,2]
    target_coordinates = f"{lat}, {lng}"

    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "type": target_type,
        "key": gkey_jr
    }

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    
    #count number of ammenities
    places_data = response.json()
    places_data
    
    #count number of ammenities in first page
    places=places_data["results"]
    names = [place["name"] for place in places]
    ammenity_count_per_page=len(names)
    ammenity_count=len(names)
    
    #print(ammenity_count_per_page)
    
    #see if more than one page of results exists
    #while "next_page_token" in places_data:
    while "next_page_token" in places_data:
        sleep(3)
        page_token=places_data["next_page_token"]
        print(page_token)
        
        params = {
        "pagetoken":page_token,
        "key": gkey_jr
        }  
        
        response = requests.get(base_url, params=params)
        places_data = response.json()
        #pprint(places_data)
        places=places_data["results"]
        names = [place["name"] for place in places]
        ammenity_count_per_page=0
        ammenity_count_per_page=len(names)
        print(ammenity_count_per_page)
        ammenity_count=ammenity_count+ammenity_count_per_page
    
    #populate dataframe
    ammenities_df['lat'].iloc[i]=lat
    ammenities_df['lng'].iloc[i]=lng
    ammenities_df['# Restaurants'].iloc[i]=ammenity_count
    ammenities_df
    
ammenities_df
i
#ammenity_count

CrQCIQEAAFsAnbbWCeRtdUf0jE_IupceGgByc5c5ZOrQY0Us5YkTYJcKvRYHxWx00mzAblYYlyCcfvV_QtWKnETZnzMaKy7Nc7kJwHd5LCh69xtNpdOyomBiBPWolhwRXq3Nxgj2XWQV1MvpUZO1s2CY05vx-iR8fVEVblca9AjZcCzSBqJcPNXq_cQZ2BpouRbbciM4OA113572QmvrZghVKW6xQSI8uu4yCbDmq0bjHx9IqLCBO1xeJlNalvcbQDBbKkYX-x6fHJXCdGQSMb9TsrZMUPANgeAA2tybgXfIzyybC9Jn16o6dkXAM3LY0pys-_rs44JWl4UgJlI6_4TWQ_tf_3fdiWUmvkA9TmuWCh2sguqFrStml14RBoBdMk9Av55nGrg93Swl7ePf0flJS63189ESECZyY2dheW3bIp7FPuTEQzMaFPJmefQf_lK3HLjxrmNDht6nOPAZ
20
CtQDwQEAAERdWPm-p0Cb-I44fNlVKoJ97fMnYCEadI_O4MlfmS85y-QgnLTlgvRl0uJECL-hDgDXmE0S-7EKEtO4TPzQ074ragyXiYlCKeOwnAXQlyBIjWPkoUfVY6DB8VTMIHYDzuoJX2_FVEdxaIUXzy3dktFJMzHTzfB1gkrWO6E8FdnrUHYeb6TB_HaPDk05LwgMokVxGY0iUHvtcKvOBYXXodM93Ifqj1GB8eyo09O1V0nbomkrui25IR4ZHGNZgFlUlH9IUE-qtCZmtJ4vjUcng8WYsz_u3WanKXK69K6btKg6C9vxEAZKV81vsGyBMSmK0CWnMb0Bcs26oLyZAvvOVQa8UEPJgi1C4HvWEY6kwC_dNX4yig0fSGYJQn7JY9-VVHxXk2g3kF4mny594qO8efw3lWtW1-tYcikdkU-XRYDBSNmSRMkMgDZT4cwcv3ZDsZcRGggC0qiWcIL2ERMOJlw11CjURZ6Y92ummx6pGols7bKHUoCwqmORe6UIpkVJTrjraLM4

6
CrQCIQEAAM6k2wCpP33s0uGEOg8IBm3Q4Tg9OaQhU_PQUJwekwxwsiyPGQn_I3zCbP_Epj6WaQZvkTVgcS__9k7A9l3goOU6yfj3cZYWrm24PKskNavteYY18SjaZ0zr3p7P8rOjWK_V7EaaraVn9RFlAK1i2_UI4TqjC9wc4oMWjCjImQ2MLX6nlSOYj_NGSFDzGGJHr4LsmQqIIN511yWZFmV3tpBZNmsNYlx2by3_O92DMJRwDjM44vMBgc4ae1-tVyTIUYvM3va565ZnIn2YBZhblnlBbpMYatJMiJv_YgD48RHSQtbIH6jlFnl_J1q4KJXfWTB-AMPer9cimmcHogGlK9j8dYQyP7jw9uIIvcvt6L2CVxLH0LDNs1k_v_l1iuZ74PnM-XBVklAaTa-0-MIUZ0gSECoyCJLTymJIBq3oRoaWzaYaFL4ihT94hr-_hjap5D0AMHLbP4aL
5
CrQCIQEAAF9SeHSiOIXXSt7o0MO-fTYTZtFGmyDuOLxeFQUw1LQ1o4Ibzq7CHMce6OkQehgimLQCJHRq2MUsytoyIDbRGncEDu_mzI65yR-FRl9iuWrefGfhjYU5pE8aKlgRggGG21rsEY_BnJUAD6m2WtFYWeqxJ_5-hb68MtLx1JtllXvD-BJtorocD7DIqmDo3fX6Z2X7TB_KWcoy2oQQLL20tBpYga8BW-7F1NRMgywdQBtJLJOUCcwanfdTHkSkYmgK5fVflrHpJ807OSuufl6LUFvuAl3ACGWDC5O2p_Zn24oqHgOu92DWwfgIxZ_sacmWQBWKxdgkuAIkcAqxKlF_HISJ5RJbxsBNPk9fZvk5WMFd2AEuP7rnvc3eiqSD3zVprjB-D_LwSJA3mm6tWEtqM3ASEMocu9wQBnx4ynDqA58J4zoaFEy1lR4W6VUwJ5JDz0__93W6qR2B
20
CtQDwQEAAFvzNOjOsLtseNYsYNRWzK7dKzUQ_Ug6KIzEu8bHRGbuQmi

18
CrQCIQEAANgZnXjzzlaUpVKMF-AgqbPEFWEkuHcQQsembZQ2-RniITJdKJWKa-dF3CHCw5cAbSeZcctkbf8UmaxeS8lFd-IQMML7MpmUvfeuB-Gzc5aW9RJCvY9GTndMVSoevDLUfKo0eJT7EClcypdGc4SFqA_3qam55KGJU7672PcAzh-xkQGIokNjJgTAtNzdv1BnjVhK0Ydm62afujn4ITvR5NQzAu_kyyU_GI2LOFad-VIYqfFxx4-n8t1kAl2sGTkD6klvz622y6TnwOaA60Vsl8VquldD7fGWOUAkEHsb135GOioGQi_KSVj7PkgkrHdvObKVUrZCsi4dZyhxitas6UtXjdiGAPvKDIZM7L1ab4-it0cubAUGwpLdj02PtKVhMEK3hUmZDucNi5B6dQhxBhsSEEAp8t1SmVz48urrltfzdbAaFADU6ivAxLyk0avrmfEXNodLkhy3
10
CrQCIQEAAJ2nqX8d0z2D3f2ywbXzYSEhSYWTKfR2DJcnldnsboAJ2zOzDWlRlHokuxXbtErtvDoBcq2_mSJu4Izttz6sCdXmQM3FJGyvXj78_K9VrDZ9bAXidGZhKGpFnPETthvXU9I3JoZe58brz1YWeo_7zlJCkbG7ciFua_DwGv6uZHnYeNl8qUWz8nx5ENe51V0adb32y3PaT2lz-3GMoPfySzKz924-ss802CCtpkOgFGmGVBUcaYO9N06iS1tjH12yuHESMk7v8R_pq7Z1qNNHPHZHHl-BorCqj314M8xEir_Xfnp-9HnXN-I8OQ6VUfwGkTzyQcDfPAsyKHAC6AUhhJZGtiIG_okNcT8Y8RUgfIVZFrF5FTPMY0zL5Lj1NVrZutCS-iy_J2jgHYGh-CAIc_4SEBsCXw2xylLTDOOfgPQh-XEaFFXiyAGrqqCdFnqLmAjM48qQtbMz
20
CtQDwQEAAARRObYchFxEEV0l1-vvbHnd7rE6CdxLr0TUOJdAVBQL5

19
CrQCIQEAAHTV9kbZCqyOoPUAVqN00Sj34WlZLvRCTysluu3RvEObz76AC90D3kJ3vyy9Mx_G5US5OIO_8_eDafAjvP7H1AqpuhOTuUcXtmlgpFNOGf_hBquNxJM9_sQch8MQnASuebh-_RDehZrnF88vURz7Mq-8nUynEwquUFPkIVhCu5MOlFLDTSJja8nrWg3KSOwPzAG_ABpF7Mndj9S4RdHNFJ6imunuP_9eHIoMz2eC66BDrypyXygdYc_k_C6Dg3frW1TQX_E7IMgzJ0gMS8_QUEAdrAmvF4Ta_rJbu5Yjg8Q0NhLquAG0aj66ak8UkaK28DP-L4U71e_hv_GaIGMZqw8jzf7xUpcpwQYk7D7KFb-q5fnRIRbkqvcG28K5jwdlHOS7JNGwZ_6e2dsfNDQ_2OUSEJDGPSUeVoC-x6M3qCy0w_saFMh5aWjAzPEQWiMf2GUoDstA8xWI
18
CrQCIQEAADzs9DuNR05oe_f5mYod0G68PM_7sP0mZCbx0_P7JYGsMzeVQtVrOfF2AL0sreY2mbWcqoIcTusxQZw4n6o7GICbdV1HyS0KvN_LWntqVivZITrQ1f5GxS2_NCXuthCncOdaKbD3RxDqeve27FGejb1RzWItJlT_MtmMjqmldE2fxxHTT9pFJEil13AbzPAts_-vTc4-x77cxvraC0i6wMVekKHoUWj81gUKY3MdYBMQ-fw7aNy-UBv0Ru7lJFKzNzvCrWj3Rs3X50ypypAoZFSIbkTB90CGGOLZd2_VZ2zmnz3ZsTILIaVFeeW2BhlNXDlwcu9gE7FxyucfrDifQSY13J0RV8fTehhY9vD3k8mer-8zYW7F_JQxAjwu-_EBfv-Dedn9Yr0gG3WL8nNEsdASECPKGuPm92eyc_utlkuEyqcaFD7kii5gam3aUdIhAw8mpTqaBRzD
2
CrQCIQEAAAQxYt66yF08wORZIgUWHd9F6nkrDDCUG9miQKAcHXdkHv

134

In [221]:
ammenities_df

,lat,lng,# Restaurants
0,29.7568,-95.3657,60
1,29.7249,-95.3638,10
2,29.7498,-95.3459,41
3,29.741,-95.3913,60
4,29.7184,-95.4236,60
5,29.7982,-95.4169,29
6,29.7715,-95.4111,60
7,29.7536,-95.3598,60
8,29.7953,-95.3676,32
9,29.7185,-95.2741,11


In [224]:
#export data to csv
output_data_file = "../data/rawdata/restaurant_count_2.csv"
ammenities_df.to_csv(output_data_file, index=False, header=True)

In [22]:
# convert response to json
places_data = response.json()

# Print the json (pretty printed)
print(json.dumps(places_data, indent=4, sort_keys=True))

{
    "error_message": "You have exceeded your daily request quota for this API. If you did not set a custom daily request quota, verify your project has an active billing account: http://g.co/dev/maps-no-account",
    "html_attributions": [],
    "results": [],
    "status": "OVER_QUERY_LIMIT"
}


In [129]:
places=places_data["results"]
places
names = [place["name"] for place in places]
ammenity_count=len(names)
ammenity_count

10